## Divide the boarder sections

In [ ]:
import numpy as np
import xarray as xr

ds = xr.open_dataset('/home/ocean_personal_data/albinski/croco_grd.nc')
mask_psi = ds['mask_psi'].values
lon_psi = ds['lon_psi'].values
lat_psi = ds['lat_psi'].values
mask_psi_bool = mask_psi == 1
midpoint = mask_psi.shape[1] // 2
midpoint_east = mask_psi.shape[0]//2

edge_mask_north_west = np.zeros_like(mask_psi_bool, dtype=bool)
edge_mask_north_west[-5:, :midpoint] = True

edge_mask_north_east = np.zeros_like(mask_psi_bool, dtype=bool)
edge_mask_north_east[-5:, midpoint:] = True

edge_mask_east_north = np.zeros_like(mask_psi_bool, dtype=bool)
edge_mask_east_north[midpoint_east:, -5:] = True

edge_mask_east_south = np.zeros_like(mask_psi_bool, dtype=bool)
edge_mask_east_south[:midpoint_east, -5:] = True

edge_mask_south_west = np.zeros_like(mask_psi_bool, dtype=bool)
edge_mask_south_west[:5, :midpoint] = True

edge_mask_south_east = np.zeros_like(mask_psi_bool, dtype=bool)
edge_mask_south_east[:5, midpoint:] = True

# Adjust the edge masks to align with smoothed_values_all dimensions
shifted_edge_mask_north_west = edge_mask_north_west[:-1, :-1]
shifted_edge_mask_north_east = edge_mask_north_east[:-1, :-1]
shifted_edge_mask_east_north = edge_mask_east_north[:-1, :-1]
shifted_edge_mask_east_south = edge_mask_east_south[:-1, :-1]
shifted_edge_mask_south_west = edge_mask_south_west[:-1, :-1]
shifted_edge_mask_south_east = edge_mask_south_east[:-1, :-1]

island_ids = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 55, 56, 57, 58, 59, 60, 100, 105, 140, 150, 200, 220, 269, 300, 330, 350]

total_values_by_island_and_section = {island_id: {'north_west': 0, 'north_east': 0, 'east_north': 0, 'east_south': 0, 'south_west': 0, 'south_east': 0} for island_id in island_ids}

for i in island_ids:
    smoothed_values_all = np.load(f'/home/ocean_personal_data/albinski/AVG_RUNS_Smoothed/avg_{i}.npy')

    for section in total_values_by_island_and_section[i]:
        if section == 'north_west':
            section_mask = shifted_edge_mask_north_west
        elif section == 'north_east':
            section_mask = shifted_edge_mask_north_east
        elif section == 'east_north':
            section_mask = shifted_edge_mask_east_north
        elif section == 'east_south':
            section_mask = shifted_edge_mask_east_south
        elif section == 'south_west':
            section_mask = shifted_edge_mask_south_west
        elif section == 'south_east':
            section_mask = shifted_edge_mask_south_east

        total_value = np.nansum(smoothed_values_all[section_mask])
        total_values_by_island_and_section[i][section] = total_value

print(total_values_by_island_and_section)


## Check masking is correct

In [ ]:
import matplotlib.pyplot as plt
ds = xr.open_dataset('/home/ocean_personal_data/albinski/croco_grd.nc')
mask_psi = ds['mask_psi'].values
lon_psi = ds['lon_psi'].values
lat_psi = ds['lat_psi'].values
f, ax = plt.subplots(1, 1, figsize=(112, 20))
plt.pcolormesh(lon_psi, lat_psi, edge_mask_east_north_east)
ax.set_aspect('equal')
plt.show()

## Raw plotting with no grouping together

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cmasher as cmr

target_islands = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 55, 56, 57, 58, 59, 60, 100, 105, 140, 150, 200, 220, 269, 300, 330, 350]

section_contributions = {island_id: {} for island_id in target_islands}
for island_id in target_islands:
    total_mass = sum(total_values_by_island_and_section[island_id].values())
    for section in ['north_west', 'north_east', 'east_north', 'east_south', 'south_west', 'south_east']:
        mass_contribution = total_values_by_island_and_section[island_id][section]
        percentage_contribution = (mass_contribution / total_mass) * 100 if total_mass > 0 else 0
        section_contributions[island_id][section] = percentage_contribution

plt.figure(figsize=(15, 8))
x_positions = np.arange(len(target_islands))
sections = ['north_west', 'north_east', 'east_north', 'east_south', 'south_west', 'south_east']
color_palette = cmr.take_cmap_colors('cmr.pride', len(sections), cmap_range=(0.1, 0.9), return_fmt='hex')

bottom = np.zeros(len(target_islands))
for idx, section in enumerate(sections):
    contributions = [section_contributions[island_id][section] for island_id in target_islands]
    plt.bar(x_positions, contributions, bottom=bottom, color=color_palette[idx], edgecolor='white', label=section)
    bottom += np.array(contributions)

plt.xlabel('Island ID')
plt.ylabel('Percentage Contribution (%)')
plt.title('Section Contribution to Each Island')
plt.xticks(x_positions, target_islands)
plt.legend(title='Section', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.ylim(0,100)

plt.show()


## Further Division of Grouping

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cmasher as cmr

target_islands = [2, 4, 5, 3, 140, 6, 7, 8, 9, 10, 11, 300, 12, 13, 14, 16, 17, 15, 18, 19, 20, 22, 23,
                105, 21, 24, 25, 33, 34, 35, 37, 42, 44, 47, 100, 
                26, 40, 27, 28, 30, 31, 29, 150, 200, 220, 269,
                46, 49, 55, 56, 57, 330, 350,
                36, 38, 39, 41, 43, 45, 48, 50, 51, 52, 53,  
                58, 59, 60]
section_contributions = {island_id: {} for island_id in target_islands}
for island_id in target_islands:
    total_mass = sum(total_values_by_island_and_section[island_id].values())
    for section in ['north_west', 'north_east', 'east_north', 'east_south', 'south_west', 'south_east']:
        mass_contribution = total_values_by_island_and_section[island_id][section]
        percentage_contribution = (mass_contribution / total_mass) * 100 if total_mass > 0 else 0
        section_contributions[island_id][section] = percentage_contribution

gap_after_islands = [140, 10, 17, 23, 100, 269, 350, 53]  
gap_width = 1.0 

x_positions = []
counter = 0
for island in target_islands:
    x_positions.append(counter)
    counter += 1
    if island in gap_after_islands:
        counter += gap_width 

plt.figure(figsize=(15, 8))
sections = ['north_west', 'north_east', 'east_north', 'east_south', 'south_west', 'south_east']
color_palette = cmr.take_cmap_colors('cmr.pride', len(sections), cmap_range=(0.1, 0.9), return_fmt='hex')

overall_bottom = np.zeros(len(x_positions))
bar_width = 1  
for idx, section in enumerate(sections):
    contributions = [section_contributions[island][section] for island in target_islands]
    plt.bar(x_positions, contributions, width=bar_width, bottom=overall_bottom, color=color_palette[idx], edgecolor='white', label=section)
    overall_bottom += np.array(contributions)

plt.ylabel('Percentage Contribution (%)')
plt.xticks(x_positions, target_islands)
plt.legend(title='Section', bbox_to_anchor=(1, 1.0125), loc='upper left')
plt.ylim(0, 100)
plt.xlim(-0.6, 74.6)


plt.show()


## Code to further divide the border into 1/4s

In [ ]:
import numpy as np
import xarray as xr

ds = xr.open_dataset('/home/ocean_personal_data/albinski/croco_grd.nc')
mask_psi = ds['mask_psi'].values
midpoint = mask_psi.shape[1] // 2
midpoint_east = mask_psi.shape[0] // 2
quarter_point = mask_psi.shape[1] // 4
quarter_point_east = mask_psi.shape[0] // 4

edge_mask_1 = np.zeros_like(mask_psi, dtype=bool)
edge_mask_1[-5:, :quarter_point] = True
edge_mask_2 = np.zeros_like(mask_psi, dtype=bool)
edge_mask_2[-5:, quarter_point:midpoint] = True
edge_mask_3 = np.zeros_like(mask_psi, dtype=bool)
edge_mask_3[-5:, midpoint:midpoint+quarter_point] = True
edge_mask_4 = np.zeros_like(mask_psi, dtype=bool)
edge_mask_4[-5:, midpoint+quarter_point:] = True

edge_mask_6 = np.zeros_like(mask_psi, dtype=bool)
edge_mask_6[midpoint_east:midpoint_east+quarter_point_east, -5:] = True
edge_mask_5 = np.zeros_like(mask_psi, dtype=bool)
edge_mask_5[midpoint_east+quarter_point_east:, -5:] = True
edge_mask_8 = np.zeros_like(mask_psi, dtype=bool)
edge_mask_8[:quarter_point_east, -5:] = True
edge_mask_7 = np.zeros_like(mask_psi, dtype=bool)
edge_mask_7[quarter_point_east:midpoint_east, -5:] = True

edge_mask_12 = np.zeros_like(mask_psi, dtype=bool)
edge_mask_12[:5, :quarter_point] = True
edge_mask_11 = np.zeros_like(mask_psi, dtype=bool)
edge_mask_11[:5, quarter_point:midpoint] = True
edge_mask_10 = np.zeros_like(mask_psi, dtype=bool)
edge_mask_10[:5, midpoint:midpoint+quarter_point] = True
edge_mask_9 = np.zeros_like(mask_psi, dtype=bool)
edge_mask_9[:5, midpoint+quarter_point:] = True

sem_1 = edge_mask_1[:-1, :-1]
sem_2 = edge_mask_2[:-1, :-1]
sem_3 = edge_mask_3[:-1, :-1]
sem_4 = edge_mask_4[:-1, :-1]

sem_6 = edge_mask_6[:-1, :-1]
sem_5 = edge_mask_5[:-1, :-1]
sem_8 = edge_mask_8[:-1, :-1]
sem_7 = edge_mask_7[:-1, :-1]

sem_12 = edge_mask_12[:-1, :-1]
sem_11 = edge_mask_11[:-1, :-1]
sem_10 = edge_mask_10[:-1, :-1]
sem_9 = edge_mask_9[:-1, :-1]

island_ids = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 55, 56, 57, 58, 59, 60, 100, 105, 140, 150, 200, 220, 269, 300, 330, 350]

total_values_by_island_and_section = {island_id: {'1': 0, '2': 0, '3': 0, '4': 0, '6': 0, '5': 0, '8': 0, '7': 0, '12': 0, '11': 0, '10': 0, '9': 0} for island_id in island_ids}

for i in island_ids:
    smoothed_values_all = np.load(f'/home/ocean_personal_data/albinski/AVG_RUNS_Smoothed/avg_{i}.npy')
    for section in total_values_by_island_and_section[i]:
        if section == '1':
            section_mask = sem_1
        elif section == '2':
            section_mask = sem_2
        elif section == '3':
            section_mask = sem_3
        elif section == '4':
            section_mask = sem_4
        elif section == '6':
            section_mask = sem_6
        elif section == '5':
            section_mask = sem_5
        elif section == '8':
            section_mask = sem_8
        elif section == '7':
            section_mask = sem_7
        elif section == '12':
            section_mask = sem_12
        elif section == '11':
            section_mask = sem_11
        elif section == '10':
            section_mask = sem_10
        elif section == '9':
            section_mask = sem_9

        total_value = np.nansum(smoothed_values_all[section_mask])
        total_values_by_island_and_section[i][section] = total_value

print(total_values_by_island_and_section)



## Divide border into 1/6ths and 1/3rds

In [ ]:
import numpy as np
import xarray as xr

ds = xr.open_dataset('/home/ocean_personal_data/albinski/croco_grd.nc')
mask_psi = ds['mask_psi'].values
sixth_point = mask_psi.shape[1] // 6
third_point_east = mask_psi.shape[0] // 3

edge_mask_1 = np.zeros_like(mask_psi, dtype=bool)
edge_mask_1[-5:, :sixth_point] = True
edge_mask_2 = np.zeros_like(mask_psi, dtype=bool)
edge_mask_2[-5:, sixth_point:2*sixth_point] = True
edge_mask_3 = np.zeros_like(mask_psi, dtype=bool)
edge_mask_3[-5:, 2*sixth_point:3*sixth_point] = True
edge_mask_4 = np.zeros_like(mask_psi, dtype=bool)
edge_mask_4[-5:, 3*sixth_point:4*sixth_point] = True
edge_mask_5 = np.zeros_like(mask_psi, dtype=bool)
edge_mask_5[-5:, 4*sixth_point:5*sixth_point] = True
edge_mask_6 = np.zeros_like(mask_psi, dtype=bool)
edge_mask_6[-5:, 5*sixth_point:] = True

edge_mask_15 = np.zeros_like(mask_psi, dtype=bool)
edge_mask_15[:5, :sixth_point] = True
edge_mask_14 = np.zeros_like(mask_psi, dtype=bool)
edge_mask_14[:5, sixth_point:2*sixth_point] = True
edge_mask_13 = np.zeros_like(mask_psi, dtype=bool)
edge_mask_13[:5, 2*sixth_point:3*sixth_point] = True
edge_mask_12 = np.zeros_like(mask_psi, dtype=bool)
edge_mask_12[:5, 3*sixth_point:4*sixth_point] = True
edge_mask_11 = np.zeros_like(mask_psi, dtype=bool)
edge_mask_11[:5, 4*sixth_point:5*sixth_point] = True
edge_mask_10 = np.zeros_like(mask_psi, dtype=bool)
edge_mask_10[:5, 5*sixth_point:] = True

edge_mask_7 = np.zeros_like(mask_psi, dtype=bool)
edge_mask_7[2*third_point_east:, -5:] = True
edge_mask_8 = np.zeros_like(mask_psi, dtype=bool)
edge_mask_8[third_point_east:2*third_point_east, -5:] = True
edge_mask_9 = np.zeros_like(mask_psi, dtype=bool)
edge_mask_9[:third_point_east, -5:] = True

sem_1 = edge_mask_1[:-1, :-1]
sem_2 = edge_mask_2[:-1, :-1]
sem_3 = edge_mask_3[:-1, :-1]
sem_4 = edge_mask_4[:-1, :-1]
sem_5 = edge_mask_5[:-1, :-1]
sem_6 = edge_mask_6[:-1, :-1]
sem_7 = edge_mask_7[:-1, :-1]
sem_8 = edge_mask_8[:-1, :-1]
sem_9 = edge_mask_9[:-1, :-1]
sem_10 = edge_mask_10[:-1, :-1]
sem_11 = edge_mask_11[:-1, :-1]
sem_12 = edge_mask_12[:-1, :-1]
sem_13 = edge_mask_13[:-1, :-1]
sem_14 = edge_mask_14[:-1, :-1]
sem_15 = edge_mask_15[:-1, :-1]

island_ids = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 55, 56, 57, 58, 59, 60, 100, 105, 140, 150, 200, 220, 269, 300, 330, 350]
total_values_by_island_and_section = {island_id: {'1': 0, '2': 0, '3': 0, '4': 0, '5': 0, '6': 0, '7': 0, '8': 0, '9': 0, '10': 0, '11': 0, '12': 0, '13': 0, '14': 0, '15': 0} for island_id in island_ids}

for i in island_ids:
    smoothed_values_all = np.load(f'/home/ocean_personal_data/albinski/AVG_RUNS_Smoothed/avg_{i}.npy')
    for section in total_values_by_island_and_section[i]:
        if section == '1':
            section_mask = sem_1
        elif section == '2':
            section_mask = sem_2
        elif section == '3':
            section_mask = sem_3
        elif section == '4':
            section_mask = sem_4
        elif section == '5':
            section_mask = sem_5
        elif section == '6':
            section_mask = sem_6
        elif section == '7':
            section_mask = sem_7
        elif section == '8':
            section_mask = sem_8
        elif section == '9':
            section_mask = sem_9
        elif section == '10':
            section_mask = sem_10
        elif section == '11':
            section_mask = sem_11
        elif section == '12':
            section_mask = sem_12
        elif section == '13':
            section_mask = sem_13
        elif section == '14':
            section_mask = sem_14
        elif section == '15':
            section_mask = sem_15

        total_value = np.nansum(smoothed_values_all[section_mask])
        total_values_by_island_and_section[i][section] = total_value

print(total_values_by_island_and_section)




## Check the border masks

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

ds = xr.open_dataset('/home/ocean_personal_data/albinski/croco_grd.nc')
lon_psi = ds['lon_psi'].values
lat_psi = ds['lat_psi'].values

composite_mask = (edge_mask_1 * 1 +
                  edge_mask_2 * 2 +
                  edge_mask_3 * 3 +
                  edge_mask_4 * 4 +
                  edge_mask_5 * 5 +
                  edge_mask_6 * 6 +
                  edge_mask_7 * 7 +
                  edge_mask_8 * 8 +
                  edge_mask_9 * 9 +
                  edge_mask_10 * 10 +
                  edge_mask_11 * 11 +
                  edge_mask_12 * 12 +
                  edge_mask_13 * 13 +
                  edge_mask_14 * 14 +
                  edge_mask_15 * 15)

f, ax = plt.subplots(1, 1, figsize=(20,5))
pcm = ax.pcolormesh(lon_psi, lat_psi, composite_mask, cmap='tab20')


ax.set_aspect('equal')
plt.show()


## Check individual border masks

In [ ]:
import matplotlib.pyplot as plt
ds = xr.open_dataset('/home/ocean_personal_data/albinski/croco_grd.nc')
mask_psi = ds['mask_psi'].values
lon_psi = ds['lon_psi'].values
lat_psi = ds['lat_psi'].values
f, ax = plt.subplots(1, 1, figsize=(20,5))

plt.pcolormesh(lon_psi, lat_psi, edge_mask_8)
plt.colorbar()
ax.set_aspect('equal')
plt.show()

## Produce edge breakdown plot from above masks

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cmasher as cmr

target_islands = [2, 4, 5, 3, 140, 6, 7, 8, 9, 10, 11, 300, 12, 13, 14, 16, 17, 15, 18, 19, 20, 22, 23,
                105, 21, 24, 25, 33, 34, 35, 37, 42, 44, 47, 100, 
                40, 26, 27, 28, 30, 31, 29, 150, 200, 220, 269,
                46, 49, 55, 56, 57, 330, 350,
                36, 38, 39, 41, 43, 45, 48, 50, 51, 52, 53,  
                58, 59, 60]
section_contributions = {island_id: {} for island_id in target_islands}
for island_id in target_islands:
    total_mass = sum(total_values_by_island_and_section[island_id].values())
    for section in ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15']:
        mass_contribution = total_values_by_island_and_section[island_id].get(section, 0)
        percentage_contribution = (mass_contribution / total_mass) * 100 if total_mass > 0 else 0
        section_contributions[island_id][section] = percentage_contribution

gap_after_islands = [140, 10, 17, 23, 100, 40, 269, 350, 53]
gap_width = 1.0

x_positions = []
counter = 0
for island in target_islands:
    x_positions.append(counter)
    counter += 1
    if island in gap_after_islands:
        counter += gap_width

plt.figure(figsize=(15, 8))
sections = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15']
color_palette = cmr.take_cmap_colors('cmr.pride', len(sections), cmap_range=(0.1, 0.9), return_fmt='hex')

overall_bottom = np.zeros(len(x_positions))
bar_width = 1
for idx, section in enumerate(sections):
    contributions = [section_contributions[island].get(section, 0) for island in target_islands]
    plt.bar(x_positions, contributions, width=bar_width, bottom=overall_bottom, color=color_palette[idx], edgecolor='white', label=section)
    overall_bottom += np.array(contributions)

plt.ylabel('Percentage of Total Risk (%)', fontsize=15)
plt.xticks(x_positions, target_islands, fontsize=8, rotation=90)
plt.yticks(fontsize=14)
plt.ylim(0, 100)
plt.xlim(-0.6, len(target_islands) + gap_width * len(gap_after_islands) - 0.4)

plt.legend(title='Sections', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.savefig('domain_map.png', dpi=1000, format='png', bbox_inches='tight')
plt.show()

